# Calculate & plot Alfven wave instability threshold

In [9]:
import numpy as np
import scipy.optimize
import scipy.interpolate
from py_vlasov.util import kzkp
# from py_vlasov.util import zp, kzkp, list_to_complex, real_imag
# from py_vlasov.wrapper import oblique_wrapper, disp_det, input_gen
# from py_vlasov.transport_ratios import transport_ratios
# from py_vlasov.follow_parameter import generate_steps, solve_disp
from KSpaceSearch import KThetaScan
import matplotlib.pyplot as plt
import matplotlib as mpl
from math import ceil
import os
import pandas as pd

mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['xtick.minor.size'] = 3.5
mpl.rcParams['ytick.minor.size'] = 3.5
mpl.rcParams['xtick.major.width'] = 1.1
mpl.rcParams['ytick.major.width'] = 1.1
mpl.rcParams['xtick.minor.width'] = .8
mpl.rcParams['ytick.minor.width'] = .8
mpl.rcParams['legend.fontsize'] = 14
#xtick.major.size     : 3.5      # major tick size in points
#xtick.minor.size     : 2      # minor tick size in points
#xtick.major.width    : 0.8    # major tick width in points
#xtick.minor.width    : 0.6    # minor tick width in points
%matplotlib inline

## Base parameters

In [4]:
# aol = np.sqrt(2.65e-8)
aol = np.sqrt(1e-8)
print('aol = ', aol)
tc_tp = 2
th_tc = 6
nc_np = 0.95
nh_np = 1 - nc_np
# vc_va = 2.42
vc_va = 5
vh_va = -vc_va * nc_np / nh_np
print('vh_va = {0:.3g}'.format(vh_va))

# betap = 0.25
betap = .5
theta = 80


t_list=[1., tc_tp, th_tc * tc_tp]
a_list=[1., 1., 1.]
n_list=[1., nc_np, nh_np] 
q_list=[1.,-1., -1.]
m_list=[1., 1./1836., 1./1836]
v_list=[0., vc_va, vh_va]
n = 6
method = 'pade'

k = 0.2

target_k = 2

kz, kp = kzkp(k, theta)

param = [kz, kp, betap, t_list, a_list, n_list, q_list,
         m_list, v_list, n, method, aol]

aol =  0.0001
vh_va = -95


In [5]:
def genParam(betap, vcva, param):
    newParam = param.copy()
    newParam[2] = betap
    n_list = param[5]
    vhva = - vcva * n_list[1] / n_list[2]
    newParam[8] = [0, vcva, vhva]
    return newParam

# data storage

In [77]:
result_db_file = 'data/alfven_instability.csv'
if os.path.exists(result_db_file):
    df = pd.read_csv(result_db_file)
else:
    column_list = ['betap', 'vcva', 'tctp', 'thtc', 'vac', 
               'max_growth_rate', 'k_peak', 'theta_peak', 
               'theta_min', 'theta_max', 'k_max']
    df = pd.DataFrame(columns= column_list)

In [78]:
df

,betap,vcva,tctp,thtc,vac,max_growth_rate,k_peak,theta_peak,theta_min,theta_max,k_max


In [71]:
# df.to_csv(result_db_file, float_format='%.3g', index=False)

In [ ]:
alfven_stop_condition = lambda wrel: (wrel.real > 1) or (wrel.imag < -1.)
ktscan = KThetaScan(param, 
                    lin_incrmt=.05, 
                    incrmt_method='linear', 
                    stop_condition=alfven_stop_condition)
theta_arr = np.linspace(80, 89.5, 19)

kz_2d, kp_2d, k_2d, wrel_2d = ktscan.fullScan(theta_arr, 0.2, 5)
ktscan.makeDiagnostics1D(theta_arr, k_2d, wrel_2d)
ktscan.makeDiagnostics2D(theta_arr, kz_2d, kp_2d, wrel_2d)